In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from tqdm.notebook import tqdm
import itertools
from plotnine import *
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Set visual style and figure size defaults
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
sns.set_palette('viridis')

analyzer = pd.read_feather("synthetic_abstracts_multi_model.feather")

analyzer.columns = analyzer.columns.str.replace('groq_', '')

data = analyzer.copy()

data.shape
data = data.dropna(subset=['abstract'])
data.shape

model = SentenceTransformer(model_name_or_path="all-MiniLM-L6-v2")

batch_size=8
embeddings = model.encode(
            data.abstract.values, 
            batch_size=batch_size,
            show_progress_bar=True,
            convert_to_tensor=True
        )

data['uid'] = data.id.astype(str) + '_' +  data.model
data.uid.duplicated().sum()

data.loc[data.diversity_params.isna(), 'diversity_params'] = {'concept_blending': '',
 'concept_granularity': '',
 'domain': '',
 'interdisciplinary_orientation': '',
 'methodological_approaches': '',
 'rhetorical_structures': '',
 'temporal_context': '',
 'terminology_density': ''}

temp = pd.json_normalize(data.diversity_params)

temp.shape
data.shape
temp['uid'] = [i for i in range(0, temp.shape[0])]
data['uid'] = [i for i in range(0, temp.shape[0])]

b = temp.merge(data, on=['uid'])
b.shape

if hasattr(embeddings, 'cpu'):
    embeddings_np = embeddings.cpu().numpy()
    

similarity_matrix = cosine_similarity(embeddings_np)

similarity_matrix

df = pd.DataFrame(similarity_matrix)

df.columns = data.uid

df.index = data.uid

# Get the indices for the upper triangle, excluding the diagonal (k=1)
indices = np.triu_indices_from(df, k=1)



# Use these indices to extract the pairs and values
item1 = df.index[indices[0]]
item2 = df.columns[indices[1]]
similarity_values = df.values[indices]

# Create the result DataFrame
df_collapsed = pd.DataFrame({'left_id': item1,
                           'right_id': item2,
                           'similarity': similarity_values})

temp.columns

comb = df_collapsed.merge(b, left_on=['left_id'], right_on=['uid'], how='left')
comb.shape

comb = comb.merge(b, left_on=['right_id'], right_on=['uid'], how='left', suffixes=['_left', '_right'])

comb['same_model'] = comb.model_left==comb.model_right
comb['same_id'] = comb.id_left==comb.id_right
comb['same_topic_spec'] = comb.topic_spec_id_left==comb.topic_spec_id_right
comb['contains_subtopics'] = comb.found_forbidden_subtopics_left==comb.found_forbidden_subtopics_right
# comb['contains_topics'] = comb.found_forbidden_topics_left==comb.found_forbidden_topics_right
comb['same_rhetoric'] = comb.rhetorical_structures_left==comb.rhetorical_structures_right
comb['same_diversity_type'] = comb.diversity_type_left==comb.diversity_type_right
comb['same_concept_blending'] = comb.concept_blending_left==comb.concept_blending_right
comb['same_concept_granularity'] = comb.concept_granularity_left==comb.concept_granularity_right
comb['same_interdisc'] = comb.interdisciplinary_orientation_left==comb.interdisciplinary_orientation_right
comb['same_methods'] = comb.methodological_approaches_left==comb.methodological_approaches_right
comb['same_temporal'] = comb.temporal_context_left==comb.temporal_context_right
comb['same_term_density'] = comb.terminology_density_left==comb.terminology_density_right
comb['same_domain'] = comb.domain_left==comb.domain_right

sample_data = comb.sample(10000)

n_samples=10000
sample_data = comb.sample(n_samples, random_state=42) # Added random_state for reproducibility

# --- Regression Analysis ---
print("Performing regression analysis...")

# Identify predictor columns (starting with 'same_')
predictor_cols = [col for col in sample_data.columns if col.startswith('same_')]

# Define dependent variable (y) and independent variables (X)
y = sample_data['similarity']
X = sample_data[predictor_cols]

# Convert boolean columns to integers (0 or 1) for regression
X = X.astype(int)

# Add a constant (intercept) to the model
X = sm.add_constant(X)

# Fit the Ordinary Least Squares (OLS) model
model_ols = sm.OLS(y, X)
results = model_ols.fit()

# --- Display Results ---
print("\n--- OLS Regression Results ---")
print(results.summary())
print("------------------------------\n")

# --- Visualize Coefficients ---
print("Generating coefficient plot...")
# Get coefficients (excluding the constant term)
coeffs = results.params.drop('const')
conf_int = results.conf_int().drop('const') # Get confidence intervals
conf_int.columns = ['lower', 'upper']
errors = (conf_int['upper'] - conf_int['lower']) / 2 # Calculate error bar lengths

# Create DataFrame for plotting
coeff_df = pd.DataFrame({
    'Coefficient': coeffs,
    'Error': errors
}).reset_index().rename(columns={'index': 'Variable'})

# Create the plot
plt.figure(figsize=(12, 7))
sns.barplot(x='Coefficient', y='Variable', data=coeff_df, palette='viridis', xerr=coeff_df['Error'])
plt.title('Regression Coefficients for Similarity Predictors (with 95% CI)')
plt.xlabel('Coefficient Value')
plt.ylabel('Predictor Variable')
plt.axvline(0, color='grey', linestyle='--') # Add line at zero
plt.tight_layout()
plt.show()

# Import necessary libraries
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer # Still useful for comparison or if needed later
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel # For accessing hidden layers
import torch # Required by transformers
from tqdm import tqdm # For progress bars

# --- Configuration ---
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
BATCH_SIZE = 16 # Adjust based on GPU memory
N_SAMPLES_REGRESSION = 10000 # Number of pairs to sample for regression
RANDOM_STATE = 42 # For reproducibility

# --- Set visual style ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
sns.set_palette('viridis')

# --- Helper Functions ---

def mean_pooling(model_output_hidden_state, attention_mask):
    """
    Performs mean pooling on token embeddings using the attention mask.

    Args:
        model_output_hidden_state: Tensor of shape (batch_size, sequence_length, hidden_size)
                                   from a specific layer.
        attention_mask: Tensor of shape (batch_size, sequence_length).

    Returns:
        Tensor of shape (batch_size, hidden_size) representing pooled embeddings.
    """
    token_embeddings = model_output_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

def get_all_layer_embeddings(texts, tokenizer, model, batch_size=8, device='cpu'):
    """
    Encodes texts and extracts pooled embeddings from ALL layers.

    Args:
        texts (list): A list of strings to encode.
        tokenizer: A Hugging Face tokenizer.
        model: A Hugging Face model enabled for hidden state output.
        batch_size (int): Processing batch size.
        device (str): 'cuda' or 'cpu'.

    Returns:
        list: A list of numpy arrays. Each array corresponds to a layer
              (0=input embeddings, 1=layer1, ...) and contains the
              pooled embeddings for all input texts for that layer.
              Shape of each array: (num_texts, hidden_size).
    """
    model.to(device)
    model.eval() # Set model to evaluation mode

    num_texts = len(texts)
    # Initialize list to store embeddings for each layer
    all_layer_pooled_embeddings = []

    print(f"Encoding {num_texts} texts in batches of {batch_size}...")
    for i in tqdm(range(0, num_texts, batch_size)):
        batch_texts = texts[i:min(i + batch_size, num_texts)]

        # Tokenize
        encoded_input = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt').to(device)

        # Compute token embeddings with hidden states
        with torch.no_grad():
            model_output = model(**encoded_input, output_hidden_states=True)

        # Get hidden states (tuple of tensors, one for each layer + input)
        # Shape: (num_layers + 1) x (batch_size, seq_len, hidden_dim)
        hidden_states = model_output.hidden_states

        # Apply mean pooling to each layer's hidden state for this batch
        attention_mask = encoded_input['attention_mask']
        if not all_layer_pooled_embeddings:
            # Initialize the list with empty lists for each layer on first batch
            all_layer_pooled_embeddings = [[] for _ in range(len(hidden_states))]

        for layer_idx, layer_hidden_state in enumerate(hidden_states):
            pooled = mean_pooling(layer_hidden_state, attention_mask)
            all_layer_pooled_embeddings[layer_idx].append(pooled.cpu().numpy())

    # Concatenate results from all batches for each layer
    final_layer_embeddings = [np.vstack(layer_batches) for layer_batches in all_layer_pooled_embeddings]

    print("Encoding complete.")
    return final_layer_embeddings


def run_regression_analysis(df_collapsed_layer, data_attributes, layer_index, n_samples, random_state):
    """
    Runs OLS regression for similarity based on 'same_*' attributes for a specific layer.

    Args:
        df_collapsed_layer (pd.DataFrame): DataFrame with 'left_id', 'right_id', 'similarity'.
        data_attributes (pd.DataFrame): DataFrame 'b' containing attributes linked by 'uid'.
        layer_index (int): The index of the layer being analyzed.
        n_samples (int): Number of pairs to sample for regression.
        random_state (int): Random state for sampling reproducibility.
    """
    print(f"\n--- Analyzing Layer {layer_index} ---")

    # --- Merge data back for comparison ---
    print(f"Layer {layer_index}: Merging data for comparison...")
    if 'uid' not in data_attributes.columns:
        print("Error: 'uid' column missing in data_attributes DataFrame 'b'. Cannot merge.")
        return

    comb = df_collapsed_layer.merge(data_attributes, left_on=['left_id'], right_on=['uid'], how='left')
    comb = comb.merge(data_attributes, left_on=['right_id'], right_on=['uid'], how='left', suffixes=['_left', '_right'])

    # --- Create 'same_*' comparison columns ---
    print(f"Layer {layer_index}: Creating 'same_*' columns...")
    
    def safe_compare(df, col_left, col_right):
        if col_left in df.columns and col_right in df.columns:
            # Handle potential NaN values before comparison if necessary
            # For simplicity here, assume columns are comparable directly
            # Fillna might be needed depending on data:
            # return df[col_left].fillna('NA') == df[col_right].fillna('NA')
            return df[col_left] == df[col_right]
        else:
            print(f"Warning: Columns {col_left} or {col_right} not found for comparison. Skipping.")
            return pd.Series([False] * len(df), index=df.index) # Ensure index alignment

    # Dynamically find all comparable columns based on suffixes
    cols_left = [col for col in comb.columns if col.endswith('_left')]
    base_cols = [col.replace('_left', '') for col in cols_left]

    for base_col in base_cols:
        col_left = base_col + '_left'
        col_right = base_col + '_right'
        # Exclude uid columns from comparison
        if base_col not in ['uid', 'diversity_params', 'uid_base', 'raw_response', 'validation_skipped', 
 'keyword_count', 'keywords_valid_format', 'found_forbidden_subtopics', 
 'abstract_length', 'abstract', 'index', 'keywords',  'found_forbidden_topics', 
 'allow_subtopic_mention' , 'prompt', 'title', 'error', 'word_count', 'provider',
'raw_response_metadata', 'topic_mix']: # Add other non-comparable base names
             # Check if both columns exist before creating the 'same_' column
             if col_left in comb.columns and col_right in comb.columns:
                  same_col_name = f'same_{base_col}'
                  comb[same_col_name] = safe_compare(comb, col_left, col_right)


    # --- Sample data ---
    actual_n_samples = min(n_samples, len(comb))
    if actual_n_samples <= 0:
        print(f"Error: Layer {layer_index}: Not enough data in 'comb' DataFrame after merging to perform sampling and regression.")
        return

    print(f"Layer {layer_index}: Sampling {actual_n_samples} data points...")
    # Check for NaN in similarity before sampling/regression
    comb_clean = comb.dropna(subset=['similarity'])
    if len(comb_clean) < actual_n_samples:
        print(f"Warning: Layer {layer_index}: Dropped {len(comb) - len(comb_clean)} rows with NaN similarity. Sampling from {len(comb_clean)} rows.")
        actual_n_samples = len(comb_clean)
        if actual_n_samples == 0:
             print(f"Error: Layer {layer_index}: No valid similarity values left after dropping NaNs.")
             return

    sample_data = comb_clean.sample(actual_n_samples, random_state=random_state)

    # --- Regression Analysis ---
    print(f"Layer {layer_index}: Performing regression analysis...")

    # Identify predictor columns (starting with 'same_')
    predictor_cols = [col for col in sample_data.columns if col.startswith('same_')]

    if not predictor_cols:
        print(f"Error: Layer {layer_index}: No 'same_*' predictor columns found.")
        return

    # Define dependent variable (y) and independent variables (X)
    y = sample_data['similarity']
    X = sample_data[predictor_cols]

    # Convert boolean columns to integers (0 or 1) for regression
    X = X.astype(int)

    # Add a constant (intercept) to the model
    X = sm.add_constant(X)

    # Check for NaN/inf values introduced (e.g., by add_constant or type conversion)
    if X.isnull().values.any() or np.isinf(X.values).any():
        print(f"Error: Layer {layer_index}: NaN or Inf values found in predictor matrix X. Check data cleaning.")
        # Optional: print X.isnull().sum() to find problematic columns
        return
    if y.isnull().values.any() or np.isinf(y.values).any():
        print(f"Error: Layer {layer_index}: NaN or Inf values found in dependent variable y.")
        return


    # Fit the Ordinary Least Squares (OLS) model
    try:
        model_ols = sm.OLS(y, X)
        results = model_ols.fit()
    except Exception as e:
        print(f"Error: Layer {layer_index}: OLS regression failed. Error: {e}")
        # Consider printing shapes: print(y.shape, X.shape)
        return

    # --- Display Results ---
    print(f"\n--- Layer {layer_index} OLS Regression Results ---")
    print(results.summary())
    print("------------------------------\n")

    # --- Visualize Coefficients ---
    print(f"Layer {layer_index}: Generating coefficient plot...")
    try:
        # Get coefficients (excluding the constant term)
        coeffs = results.params.drop('const')
        conf_int = results.conf_int().drop('const') # Get confidence intervals
        conf_int.columns = ['lower', 'upper']
        # Calculate error bar lengths (handle potential NaN in CI)
        errors = (conf_int['upper'] - conf_int['lower']) / 2
        errors = errors.fillna(0) # Replace NaN errors with 0 for plotting

        # Create DataFrame for plotting
        coeff_df = pd.DataFrame({
            'Coefficient': coeffs,
            'Error': errors
        }).reset_index().rename(columns={'index': 'Variable'})

        # Create the plot
        plt.figure(figsize=(12, max(7, len(coeff_df) * 0.5))) # Adjust height based on number of variables
        sns.barplot(x='Coefficient', y='Variable', data=coeff_df, palette='viridis', xerr=coeff_df['Error'])
        plt.title(f'Layer {layer_index}: Regression Coefficients for Similarity Predictors (with 95% CI)')
        plt.xlabel('Coefficient Value')
        plt.ylabel('Predictor Variable')
        plt.axvline(0, color='grey', linestyle='--') # Add line at zero
        plt.tight_layout()
        plt.show()
    except Exception as e:
        print(f"Error: Layer {layer_index}: Failed to generate coefficient plot. Error: {e}")

In [ ]:
# --- Main Execution ---

# Load data (assuming the file exists in the environment)
analyzer = pd.read_feather("synthetic_abstracts_multi_model.feather")


# Clean column names
analyzer.columns = analyzer.columns.str.replace('groq_', '')

# Copy data and handle missing abstracts
data = analyzer.copy()
data = data.dropna(subset=['abstract'])
print(f"Loaded {len(data)} rows after dropping NA abstracts.")

if len(data) < 2:
    print("Error: Not enough data rows (< 2) to proceed with analysis.")
    # Exit or handle appropriately
    exit()

# --- Prepare Attribute Data ('b') ---
print("Preparing attribute data...")
# Create unique IDs
data['uid_base'] = data.id.astype(str) + '_' + data.model.astype(str)

# Handle potential duplicate UIDs more robustly - use index if needed
if data.uid_base.duplicated().any():
    print("Warning: Duplicate base UIDs (id + model) detected. Using DataFrame index as unique ID ('uid').")
    data = data.reset_index(drop=True)
    data['uid'] = data.index.astype(str)
else:
    data['uid'] = data['uid_base']

# Handle missing diversity_params
default_params = {
    'concept_blending': '', 'concept_granularity': '', 'domain': '',
    'interdisciplinary_orientation': '', 'methodological_approaches': '',
    'rhetorical_structures': '', 'temporal_context': '', 'terminology_density': ''
}
data['diversity_params'] = data['diversity_params'].apply(lambda x: x if isinstance(x, dict) else default_params)

# Normalize diversity_params
temp = pd.json_normalize(data['diversity_params'])

# Ensure temp and data align using index reset (important!)
data = data.reset_index(drop=True)
temp = temp.reset_index(drop=True)

# Merge normalized params back with main data
b = pd.concat([data, temp], axis=1)
print(f"Attribute data 'b' prepared with shape: {b.shape}")

# --- Load Model and Tokenizer ---
print(f"Loading tokenizer and model: {MODEL_NAME}...")
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model_hf = AutoModel.from_pretrained(MODEL_NAME)
    # Determine device
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")
except Exception as e:
    print(f"Error loading model or tokenizer: {e}")
    exit() # Cannot proceed without model

# --- Get Embeddings for All Layers ---
all_embeddings_list = get_all_layer_embeddings(
    data.abstract.tolist(), # Use the abstract list from the final 'data' df
    tokenizer,
    model_hf,
    batch_size=BATCH_SIZE,
    device=device
)
# all_embeddings_list contains numpy arrays: [layer0_embeds, layer1_embeds, ...]
# Number of layers = len(all_embeddings_list) - 1 (excluding input embedding layer 0)
num_layers_output = len(all_embeddings_list)
print(f"Extracted embeddings for {num_layers_output} layers (including input).")

# --- Loop Through Layers for Analysis ---
# Ensure the number of embeddings matches the attribute data 'b'
# This check should ideally be inside the embedding function or handled robustly
if all_embeddings_list and all_embeddings_list[0].shape[0] != b.shape[0]:
     print(f"FATAL ERROR: Mismatch between number of embeddings ({all_embeddings_list[0].shape[0]}) and attribute data rows ({b.shape[0]}). Aborting.")
     # This indicates a critical issue in data preparation or embedding steps.
     exit()

for i, layer_embeddings_np in enumerate(all_embeddings_list):
    print(f"\n=======================================")
    print(f"Processing Layer {i} Embeddings...")
    print(f"=======================================")

    # --- Calculate Similarity Matrix ---
    print(f"Layer {i}: Calculating similarity matrix...")
    similarity_matrix = cosine_similarity(layer_embeddings_np)
    # Ensure index/columns align with the 'uid' in 'b'
    df_sim = pd.DataFrame(similarity_matrix, index=b['uid'], columns=b['uid'])

    # --- Collapse Similarity Matrix ---
    print(f"Layer {i}: Collapsing similarity matrix...")
    indices = np.triu_indices_from(df_sim, k=1) # Exclude diagonal

    # Check if indices are valid (matrix might be too small)
    if len(indices[0]) == 0:
        print(f"Warning: Layer {i}: No pairs found in the upper triangle (matrix size likely <= 1). Skipping analysis for this layer.")
        continue

    item1 = df_sim.index[indices[0]]
    item2 = df_sim.columns[indices[1]]
    similarity_values = df_sim.values[indices]

    df_collapsed_layer = pd.DataFrame({'left_id': item1,
                                       'right_id': item2,
                                       'similarity': similarity_values})

    # --- Run Regression for this Layer ---
    run_regression_analysis(df_collapsed_layer, b, i, N_SAMPLES_REGRESSION, RANDOM_STATE)


print("\n--- Layer-wise Analysis Complete ---")